
# Import Data

Importing AXP data.

In [2]:

import pandas as pd

amex_df = pd.read_csv('../../data/gen/usAXP_df.csv',
                           parse_dates=True,
                           index_col=0)
# Strip whitespace from column names
amex_df.columns = amex_df.columns.str.strip()
# Print
amex_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key
,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4



## Processing Data

Adding slope & velocity to each day.

In [3]:

import numpy as np

extra_df = amex_df.copy()
extra_df['slope'] = pd.Series(np.gradient(extra_df.close), extra_df.index, name='slope')
extra_df['velocity'] = pd.Series(np.gradient(extra_df.slope), extra_df.index, name='velocity')
# Print
extra_df.head()


,close,high,low,p_change,open,pre_close,volume,date,date_week,atr21,atr14,key,slope,velocity
,,,,,,,,,,,,,,
2017-06-12,80.17,80.73,79.95,-0.174,80.35,80.31,3352279,20170612,0,0.780000,0.780000,0,0.420,-0.0850
2017-06-13,80.59,80.74,80.07,0.524,80.20,80.17,3174361,20170613,1,0.722381,0.721071,1,0.335,-0.1825
2017-06-14,80.84,80.92,79.62,0.310,80.11,80.59,4013089,20170614,2,0.933535,0.942224,2,0.055,-0.0150
2017-06-15,80.70,81.24,80.23,-0.173,80.38,80.84,2773369,20170615,3,0.955464,0.962959,3,0.305,0.2675
2017-06-16,81.45,81.48,80.77,0.929,80.86,80.70,5914676,20170616,4,0.896598,0.896962,4,0.590,0.1125



# Performing Backtests

1. Prepare Parameters
2. Define Backtest Tasks
3. Defining a DataFrame to handle results
4. Running BackTest on Multi Threads

## Preparing Parameters


In [13]:

import itertools

# list_buy_slope_threshold = np.arange(-0.1, 0.1, 0.1)
# list_buy_velocity_threshold = np.arange(-0.1, 0.1, 0.1)
# list_close_slope_threshold = np.arange(-0.1, 0.1, 0.1)
# list_close_velocity_threshold = np.arange(-0.1, 0.1, 0.1)
# list_sell_slope_threshold = np.arange(-0.1, 0.1, 0.1)
# list_sell_velocity_threshold = np.arange(-0.1, 0.1, 0.1)

list_buy_slope_threshold = np.arange(-0.5, 0.5, 0.1)
list_buy_velocity_threshold = np.arange(-0.5, 0.5, 0.1)
list_close_slope_threshold = np.arange(-0.2, 0.2, 0.1)
list_close_velocity_threshold = np.arange(-0.2, 0.2, 0.1)
list_sell_slope_threshold = np.arange(-0.5, 0.5, 0.1)
list_sell_velocity_threshold = np.arange(-0.5, 0.5, 0.1)

task_list = list(itertools.product(
    list_buy_slope_threshold, 
    list_buy_velocity_threshold, 
    list_close_slope_threshold, 
    list_close_velocity_threshold, 
    list_sell_slope_threshold, 
    list_sell_velocity_threshold
))
print("Params Ready, {} tasks to run, will take approximately {} minutes or {} hours to complete. ".format(len(task_list), round(len(task_list)/(5*60), 2), round(len(task_list)/(5*60*60), 2)))


Params Ready, 160000 tasks to run, will take approximately 533.33 minutes or 8.89 hours to complete. 



# Defining Backtest Task


In [5]:

import import_ipynb
from strategy_base import TradeStrategyBase

def backtest(tradedays: pd.DataFrame, 
             strategy: TradeStrategyBase):
    
    # Iterate through each data and trade
    for date, trade_day in extra_df.iterrows():
        strategy.trade(date=date,
                       tradeday=trade_day)
    # Return resulting profit
    return strategy


importing Jupyter notebook from strategy_base.ipynb



## Defining a DataFrame to handle results


In [6]:

final_result = pd.DataFrame(columns=['buy_slope_threshold', 'buy_velocity_threshold', 'close_slope_threshold', 'close_velocity_threshold', 'sell_slope_threshold', 'sell_velocity_threshold', 'profit_percentage'])
final_result


,buy_slope_threshold,buy_velocity_threshold,close_slope_threshold,close_velocity_threshold,sell_slope_threshold,sell_velocity_threshold,profit_percentage



## Diagnosing Environment
    

In [7]:

import multiprocessing

print("System is {} cpus".format(multiprocessing.cpu_count()))


System is 8 cpus



## Running Backtest with Multi-Threading


In [8]:

import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures.process import ProcessPoolExecutor
from tqdm import tqdm
from strategy_4 import TradeStrategy4

with ProcessPoolExecutor() as executor:
    """
    By default, ProcessPool uses maximum available number of cores to process.
    """
    
    """
    Equivalent to executor.map(fn, *iterables),
    but displays a tqdm-based progress bar.
    
    Does not support timeout or chunksize as executor.submit is used internally
    
    **kwargs is passed to tqdm.
    """
    futures_list = []
    kwargs = {
        'total': len(futures_list),
        'unit': 'tests',
        'unit_scale': True,
        'leave': True
    }
        
    futures_list = [executor.submit(backtest, 
                                    extra_df,
                                    TradeStrategy4(buy_slope_threshold=buy_slope_threshold,
                                                   buy_velocity_threshold=buy_velocity_threshold,
                                                   close_slope_threshold=close_slope_threshold,
                                                   close_velocity_threshold=close_velocity_threshold,
                                                   sell_slope_threshold=sell_slope_threshold,
                                                   sell_velocity_threshold=sell_velocity_threshold,
                                                   should_log=False, 
                                                   should_plot=False)) for buy_slope_threshold, \
                                                                           buy_velocity_threshold, \
                                                                           close_slope_threshold, \
                                                                           close_velocity_threshold, \
                                                                           sell_slope_threshold, \
                                                                           sell_velocity_threshold in task_list]
    print("Starting {} tasks".format(len(futures_list)))
    
    for f in tqdm(concurrent.futures.as_completed(futures_list), **kwargs):
        # done_callback runs on the main process
        strategy = f.result()
        
        result_row = [strategy.buy_slope_threshold, 
                      strategy.buy_velocity_threshold, 
                      strategy.close_slope_threshold, 
                      strategy.close_velocity_threshold, 
                      strategy.sell_slope_threshold, 
                      strategy.sell_velocity_threshold, 
                      strategy.trade_profit * 100]
        
        final_result = final_result.append(pd.Series(result_row, index=final_result.columns), ignore_index=True)


importing Jupyter notebook from strategy_4.ipynb
Starting 4096 tasks


4.10ktests [12:47, 5.34tests/s]



# Visualizing Results


In [9]:
        
final_result.sort_values(by=['profit_percentage'])
final_result.head(100)


,buy_slope_threshold,buy_velocity_threshold,close_slope_threshold,close_velocity_threshold,sell_slope_threshold,sell_velocity_threshold,profit_percentage
0,-0.2,-0.2,-0.2,-0.2,-0.2,-0.2,206.205450
1,-0.2,-0.2,-0.2,-0.2,-0.2,-0.1,206.205450
2,-0.2,-0.2,-0.2,-0.2,-0.1,-0.2,206.205450
3,-0.2,-0.2,-0.2,-0.2,-0.2,0.0,206.205450
4,-0.2,-0.2,-0.2,-0.2,-0.2,0.1,206.205450
5,-0.2,-0.2,-0.2,-0.2,-0.1,-0.1,206.205450
6,-0.2,-0.2,-0.2,-0.2,-0.1,0.0,206.205450
7,-0.2,-0.2,-0.2,-0.2,-0.1,0.1,206.205450
8,-0.2,-0.2,-0.2,-0.2,0.0,-0.2,206.205450
9,-0.2,-0.2,-0.2,-0.1,-0.2,-0.2,207.044979
